In [1]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
from torch.nn.modules.loss import *
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
from pathlib import Path
# %matplotlib notebook

/home/drake/anaconda3/envs/.torch/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/drake/anaconda3/envs/.torch/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/drake/anaconda3/envs/.torch/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/drake/anaconda3/envs/.

In [2]:
from Loss.triplet import *
from session import *
from LR_Schedule.cyclical import Cyclical
from LR_Schedule.cos_anneal import CosAnneal
from LR_Schedule.lr_find import lr_find
from callbacks import *
from validation import *
from validation import _AccuracyMeter
import Datasets.ImageData as ImageData
from Transforms.ImageTransforms import *
import util
import Datasets.ModelData as md

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
torch.cuda.set_device(0); torch.backends.cudnn.benchmark=True;

/home/drake/anaconda3/envs/.torch/lib/python3.7/site-packages/torch/cuda/__init__.py:135: UserWarning: 
    Found GPU0 GeForce GTX 770 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability that we support is 3.5.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


In [5]:
data_path = Path("/media/drake/MX500/Datasets/Kannada-MNIST")
train_path = data_path/'train.csv'
test_path = data_path/'test.csv'
df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [6]:
labels = df[df.columns[0]].to_numpy()
inputs = df[df.columns[1:]].to_numpy().reshape(-1, 28, 28)

ids = test_df[test_df.columns[0]].to_numpy()
test_inputs = test_df[test_df.columns[1:]].to_numpy().reshape(-1, 28, 28)

inputs.shape, labels.shape

((60000, 28, 28), (60000,))

In [7]:
class KannadaMNISTDataset(Dataset):
    def __init__(self, inputs, labels, transform):
        self.data = inputs
        self.targets = labels
        self.tsfm = transform
        
    def __len__(self): return self.targets.shape[0]
        
    def __getitem__(self, i):
        x, y = self.data[i], self.targets[i]  
        x = x.astype(np.uint8)
        x = self.tsfm(x)
        x = x.float()
        
        return x, y

In [8]:
i_dict = md.make_partition_indices(labels.shape[0], {'train': .9, 'valid': .1})

In [9]:
transform = transforms.Compose([transforms.ToPILImage(),
                                transforms.RandomRotation(9),
                                transforms.RandomResizedCrop(28, scale=(.95, 1.05)),
                                transforms.ToTensor(), 
                                transforms.Normalize((0.5,), (0.5,))])

train_dataset = KannadaMNISTDataset(inputs[i_dict['train']], labels[i_dict['train']], transform)
valid_dataset = KannadaMNISTDataset(inputs[i_dict['valid']], labels[i_dict['valid']], transform)
test_dataset = KannadaMNISTDataset(test_inputs, ids, transform)
trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valloader = DataLoader(valid_dataset, batch_size=64, shuffle=False)
testloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [10]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

## Baseline

In [ ]:
model_base = nn.Sequential(
    nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(16),
    nn.ReLU(),
    
    nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
     
    nn.MaxPool2d(kernel_size=2, stride=2),
    
    nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    
    nn.MaxPool2d(kernel_size=2, stride=2),
    
    nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    
    nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    
    nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    
    Flatten(),
    
    nn.Linear(7 * 7 * 32, 50),
    nn.Linear(50, 25),
    nn.Linear(25, 10)
    # nn.LogSoftmax()
)

sess_base = Session(model_base, nn.CrossEntropyLoss(), optim.Adam, 1e-3)

In [ ]:
lr_find(sess_base, trainloader, start_lr=1e-7)

In [ ]:
sess_base.set_lr(1e-3)

In [ ]:
validator = Validator(valloader, OneHotAccuracy(), save_best=True, model_dir='./base')
lr_scheduler = CosAnneal(len(trainloader), T_mult=2)
schedule = TrainingSchedule(trainloader, [lr_scheduler, validator])
sess_base.train(schedule, 7)

In [ ]:
def inference_test_data(model, submission_path=None):
    if submission_path is None:
        submission_path = Path('./Kannada.submission.csv')
        
    submission = []
    
    model = util.to_gpu(model)
    
    with EvalModel(model):       
        for input, id in testloader:
            outputs = model(util.to_gpu(input))
            _, preds = torch.max(outputs, 1)
            

            batch = torch.stack([id, preds.cpu()], dim=1).numpy()          

            submission.append(batch)

    submission = np.concatenate(submission)
   
    df = pd.DataFrame(submission, columns=['id', 'label'])
    
    df.to_csv(submission_path, index=False)
        
    return df

In [ ]:
sub = inference_test_data(model_base)
sub

## Triplet Loss 


In [11]:
class TripletRegularizedCrossEntropyLoss(nn.Module):
    def __init__(self, alpha, margin):     
        super().__init__()
        self.alpha = alpha
        self.margin = margin
        
    def forward(self, x, y):
        loss = F.cross_entropy(x[-1], y)
        triplet = 0
        
        for layer in x[:-1]:
            triplet += batch_hard_triplet_loss(layer.view(layer.size(0), -1), y, self.margin)
            
        # triplet *= min(self.alpha/math.sqrt(loss.item()), 1)
        triplet *= self.alpha
            
        return loss + triplet
    
    
class CustomOneHotAccuracy(OneHotAccuracy):
    def __init__(self):
        super().__init__()
        self.reset()

    def update(self, output, label):
        super().update(output[-1], label)
        
class SelectiveSequential(nn.Module):
    def __init__(self, to_select, modules_dict):
        super(SelectiveSequential, self).__init__()
        for key, module in modules_dict.items():
            self.add_module(key, module)
        self._to_select = to_select
    
    def forward(self, x):
        list = []
        for name, module in self._modules.items():               
            x = module(x)      
            
            if name in self._to_select:
                list.append(x)
                
        return list
    

In [12]:
model = SelectiveSequential(
    ['act64a', 'act128', 'act64b', 'act32b', 'fc1', 'fc2', 'out'],
    {'conv16': nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
     'batch16': nn.BatchNorm2d(16),
     'act16': nn.ReLU(),
     
     'drop1': nn.Dropout(.12),
    
     'conv32a': nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
     'batch32a': nn.BatchNorm2d(32),
     'act32a': nn.ReLU(),
     
     'drop1': nn.Dropout(.15),
     
     'max1': nn.MaxPool2d(kernel_size=2, stride=2),
    
     'conv64a': nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
     'batch64a': nn.BatchNorm2d(64),
     'act64a': nn.ReLU(),
    
     'max2': nn.MaxPool2d(kernel_size=2, stride=2),
    
     'conv128': nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2),
     'batch128': nn.BatchNorm2d(128),
     'act128': nn.ReLU(),
     
     'drop1': nn.Dropout(.2),
    
     'conv64b': nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
     'batch64b': nn.BatchNorm2d(64),
     'act64b': nn.ReLU(),
     
     'drop1': nn.Dropout(.15),
    
     'conv32b': nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),
     'batch32b': nn.BatchNorm2d(32),
     'act32b': nn.ReLU(),
     
     'drop1': nn.Dropout(.12),
    
     'flatten': Flatten(),
    
     'fc1': nn.Linear(7 * 7 * 32, 50),
     'drop1': nn.Dropout(.05),
     'fc2': nn.Linear(50, 25),
     'drop1': nn.Dropout(.05),
     'out': nn.Linear(25, 10)})

In [13]:
criterion = TripletRegularizedCrossEntropyLoss(.25, 1)
sess = Session(model, criterion, optim.AdamW, 1e-3)

In [ ]:
lr_find(sess, trainloader, start_lr=1e-7)

In [14]:
sess.set_lr(2e-4)

In [ ]:
lr_scheduler.plot(len(trainloader) * 4)

In [15]:
validator = Validator(valloader, CustomOneHotAccuracy(), save_best=True, model_dir='./triplet')
lr_scheduler = CosAnneal(len(trainloader), T_mult=2)
schedule = TrainingSchedule(trainloader, [lr_scheduler, validator])
sess.train(schedule, 63)

Validating: 100%|██████████| 94/94 [00:02<00:00, 32.69it/s]


Training Loss: 1.408563494682312  Validaton Loss: 1.5481818914413452 Validation Accuracy: 0.9866666666666666


Validating: 100%|██████████| 94/94 [00:02<00:00, 33.01it/s]


Training Loss: 0.9349409937858582  Validaton Loss: 0.9910321235656738 Validation Accuracy: 0.9911666666666666


Validating: 100%|██████████| 94/94 [00:02<00:00, 32.83it/s]


...

Training Loss: 0.13694842159748077  Validaton Loss: 0.19868730008602142 Validation Accuracy: 0.9971666666666666


Validating: 100%|██████████| 94/94 [00:02<00:00, 32.96it/s]

Training Loss: 0.15716391801834106  Validaton Loss: 0.23361383378505707 Validation Accuracy: 0.997


Validating: 100%|██████████| 94/94 [00:02<00:00, 32.58it/s]

Training Loss: 0.15016405284404755  Validaton Loss: 0.21592000126838684 Validation Accuracy: 0.9958333333333333



sess2 = Session(model, TripletRegularizedCrossEntropyLoss(0, 1), optim.AdamW, 1e-4)
validator = Validator(valloader, CustomOneHotAccuracy(), save_best=True, model_dir='./triplet')
lr_scheduler = CosAnneal(len(trainloader), T_mult=2)
schedule = TrainingSchedule(trainloader, [lr_scheduler, validator])
sess2.train(schedule, 7)